In [26]:
from shapely.geometry import Point
from math import acos, cos, sin, radians, asin, sqrt
import GPSPreprocess as gpsp
import pytest
import geopandas as gpd
import pandas as pd
import os
import sys
import re
import datetime
from math import acos, cos, sin, radians, asin, sqrt, floor
import GPSPreprocess as gpsp
import pytest
import geopandas as gpd
import pandas as pd
import os
import sys
import re
import datetime

sample_gps_file_path = os.getcwd().split(
"PyERT-BLACK")[0] + 'PyERT-BLACK/quarto-example/data/sample-gps'

sample_gps = f'{sample_gps_file_path}/sample-gps-1.csv'
Data = pd.read_csv(sample_gps)
DataLength = len(Data)
GPSData = gpsp.GPSPreprocess(data=Data)

ProcessedData = GPSData.getData()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(ProcessedData.head(1000))


        
def distance(cur, next):
    lon1 = radians(cur.x)
    lon2 = radians(next.x)
    lat1 = radians(cur.y)
    lat2 = radians(next.y)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6378137
    return c*r
seconds = []
acc = []
zeros = []
marker = []
segment_time = 0
last_index = len(ProcessedData.index) - 1
for index,row in ProcessedData.iterrows():
    curr_min = row["LocalTime"][-2:]
    if (index == 0):
        start_min = curr_min
    elif (curr_min != start_min):
        # if no zeros in segment, no need to do anything
        if (zeros):
            # inaccuracies in measurements can lead to negative time
            avg_missing_time = (max(59.99, segment_time) - segment_time) / len(zeros)
            for i in zeros:
                seconds[i] = avg_missing_time
                cur_speed = 0
                next_speed = ProcessedData[i+1:i+2]['Speed_kmh'].iloc[0]*5/18
                #print(f'cur_speed={cur_speed}, next_speed={next_speed}, time={avg_missing_time}, acc={abs(cur_speed - next_speed) / avg_missing_time}')
                if (avg_missing_time != 0):
                    acc[i] = abs(cur_speed - next_speed) / avg_missing_time
                
                
        start_min = curr_min
        segment_time = 0
        zeros = []
        marker.append(index)

    cur_speed = row['Speed_kmh']*5/18
    if (cur_speed == 0):
        seconds.append(0)
        zeros.append(index)
        acc.append(0)
        continue
    cur = row['geometry']
    next = ProcessedData[index+1:index+2]['geometry']
    time = distance(cur, next)/(cur_speed)
    seconds.append(time)
    segment_time = segment_time + time
    if (index == last_index):
        acc.append(0)
    else:
        next_speed = ProcessedData[index+1:index+2]['Speed_kmh'].iloc[0]*5/18
        acc.append(abs(cur_speed - next_speed) / time)
    
# summing time in each minute segment
sum = 0
mark = marker.pop(0)
for i in range(len(seconds)):
    if (i == mark):
        sum = 0
        if (marker):
            mark = marker.pop(0)
    sum += seconds[i]
    seconds[i] = sum

stage_points = {}
prev_time = 0
episode_len = 0
episode_lens = []
temp_episode_len = 0
temp_stop_episode_len = 0
start_index = 0
ep_id= 0
last_car_index = -1
valid_stop = True
last_walk_index = -1


last = len(ProcessedData.index) - 1
for index,row in ProcessedData.iterrows():
    speed = float(row['Speed_kmh'])
    #print(speed)
    if (0.36<= speed <= 10.008):
        new_mode = 'walk' 
    elif (speed < 0.36): 
        new_mode = 'stop'
    elif (speed > 10.008):
        new_mode = 'car'

    pattern = '(\d+)\/(\d+)\/(\d+)\s(\d+):(\d\d)'
    match = re.search(pattern, row['LocalTime'])
    # inaccuracies in distance and speed can cause impossible times
    second = min(floor(seconds[index]), 59)
    microseconds = min(floor((seconds[index] - second)*10**6), 999999)
    cur_time = datetime.datetime(second=second, microsecond=microseconds, year = int(match.groups()[2]),month = int(match.groups()[0]), day = int(match.groups()[1]), hour = int(match.groups()[3]), minute = int(match.groups()[4]))
    if index == 0:
        cur_mode = new_mode
        prev_time = cur_time
    time_difference = (cur_time - prev_time).total_seconds()
    prev_time = cur_time
    episode_len += time_difference
    episode_lens.append(episode_len)
    
    if (time_difference >= 120): 
        #print(f'GAP time_difference: {episode_len} cur_mode: {cur_mode}')
        if (len(episode_lens) > 2 and ((cur_mode == 'car' and episode_lens[index-2] >= 120) or (cur_mode == 'walk' and [index-2] >= 60 ) or (cur_mode == 'stop' and [index-2] >= 120))):
            ep_id +=1
            stage_points[ep_id] = {}
            stage_points[ep_id]['start'] = start_index
            stage_points[ep_id]['end'] = index-2
            stage_points[ep_id]['mode'] = cur_mode
        else:
            stage_points[ep_id]['end'] = index-2

        ep_id+=1
        stage_points[ep_id] = {}
        stage_points[ep_id]['start'] = index - 1
        stage_points[ep_id]['end'] = index
        stage_points[ep_id]['mode'] =  'stop'

        episode_len = 0
        start_index = index + 1
        cur_mode = 'stop'
    elif (index == last):
        if ((cur_mode == 'car' and episode_len >= 120) or (cur_mode == 'walk' and episode_len >= 60 ) or (cur_mode == 'stop' and episode_len >= 120)):
            ep_id +=1
            stage_points[ep_id] = {}
            stage_points[ep_id]['start'] = start_index
            stage_points[ep_id]['end'] = index
            stage_points[ep_id]['mode'] = cur_mode
    else:
        if (new_mode != cur_mode):
            if (new_mode != 'car' and cur_mode == 'car'): #currently, no invalid car episodes
                if (last_car_index < 0):
                    last_car_index = index - 1
                    temp_episode_len = 0

                if (speed >= 0.36): 
                    valid_stop = False

                temp_episode_len += time_difference
                #print('temp_episode_len: ' + str(temp_episode_len))
                if ( (not valid_stop and temp_episode_len >= 120) or (valid_stop and temp_episode_len >= 60)):
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = last_car_index
                    stage_points[ep_id]['mode'] = 'car'
                    start_index = last_car_index + 1
                    last_car_index = -1
                    episode_len = temp_episode_len
                    cur_mode = new_mode
                    valid_stop = True
                
            elif (new_mode == 'stop' and cur_mode == 'walk'): 
                if (last_walk_index < 0):
                    last_walk_index = index - 1
                    temp_stop_episode_len = 0

                temp_stop_episode_len += time_difference
                #print('temp_episode_len: ' + str(temp_episode_len))
                if (temp_stop_episode_len >= 120):
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = last_walk_index
                    stage_points[ep_id]['mode'] = 'walk'
                    start_index = last_walk_index + 1
                    last_walk_index = -1
                    episode_len = temp_stop_episode_len
                    cur_mode = new_mode
            

            elif ((new_mode !='walk' and cur_mode == 'walk') or (new_mode != 'stop' and cur_mode == 'stop')) :  
                if ((cur_mode == 'walk' and episode_len >= 60) or (cur_mode == 'stop' and episode_len >= 120) ):
                    #print(f'valid episode ep_len: {episode_len} cur_mode: {cur_mode} new_mode: {new_mode}')
                    ep_id +=1
                    stage_points[ep_id] = {}
                    stage_points[ep_id]['start'] = start_index
                    stage_points[ep_id]['end'] = index - 1
                    stage_points[ep_id]['mode'] = cur_mode
                else :
                    #print(f'invalid episode')
                    if (ep_id == 0):
                        stage_points[ep_id] = {}
                        stage_points[ep_id]['start'] = start_index
                        stage_points[ep_id]['end'] = index - 1
                        stage_points[ep_id]['mode'] = 'invalid'
                    if (ep_id != 0):
                        stage_points[ep_id]['end'] = index - 1
                
                start_index = index
                episode_len = 0
                cur_mode = new_mode
        else:
            if temp_episode_len > 0:
                last_car_index = -1
                temp_episode_len = 0
                valid_stop = True
            if temp_stop_episode_len > 0:
                last_walk_index = -1
                temp_stop_episode_len = 0

serial_ids = []
record_ids = []
start_times = []
modes = []
geometry = []

invalid_first_stage = False

for key in stage_points:
    #print(f"key: {key} start: {stage_points[key]['start']} end: {stage_points[key]['end']} mode: {stage_points[key]['mode']}")
    if stage_points[key]['mode'] == 'invalid':
        invalid_first_stage = True
        target = ProcessedData.iloc[stage_points[key]['start']]
    else:
        if (not invalid_first_stage):
            target = ProcessedData.iloc[stage_points[key]['start']]
        else:
            invalid_first_stage = False
        serial_ids.append(target.SerialID)
        record_ids.append(target.RecordID)
        start_times.append(target.LocalTime)
        modes.append(stage_points[key]['mode'])
        geometry.append(target.geometry)

data = { 'EPISODEID' : serial_ids, 
        'RECORDID' : record_ids, 
        'TIMESTART' : start_times, 
        'MODES' : modes,
        'geometry': geometry}
gdf = gpd.GeoDataFrame(data)

print(gdf)
# extraction
trip = gdf[gdf["MODES"].isin(['walk', 'car'])]
start = 0
startIndex = 0
dropped = []
for index,row in trip.iterrows():
    point = row["geometry"] 
    if (index == 0):
        start = point
        startIndex = index
        continue
    # do not want to drop last row even if it violates the min distance
    if (index == trip.shape[0]):
        break
    lon1 = radians(start.x)
    lon2 = radians(point.x)
    lat1 = radians(start.y)
    lat2 = radians(point.y)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6378137
      
    # calculate the result
    distance=c*r
    #print(f'start={startIndex}, end={index}, distance={distance}')
    if (distance < 5):
        dropped.append(index)
    else:
        startIndex = index
        start = point

print(trip)
# def distance(cur, next):
#     lon1 = radians(cur.x)
#     lon2 = radians(next.x)
#     lat1 = radians(cur.y)
#     lat2 = radians(next.y)
#     dlon = lon2 - lon1
#     dlat = lat2 - lat1
#     a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#     c = 2 * asin(sqrt(a))
#     r = 6378137
#     return c*r
# seconds = []
# acc = []
# zeros = []
# marker = []
# segment_time = 0
# last_index = len(ProcessedData.index) - 1
# for index,row in ProcessedData.iterrows():
#     curr_min = row["LocalTime"][-2:]
#     if (index == 0):
#         start_min = curr_min
#     elif (curr_min != start_min):
#         # if no zeros in segment, no need to do anything
#         if (zeros):
#             # inaccuracies in measurements can lead to negative time
#             avg_missing_time = (max(59.99, segment_time) - segment_time) / len(zeros)
#             for i in zeros:
#                 seconds[i] = avg_missing_time
#                 cur_speed = 0
#                 next_speed = ProcessedData[i+1:i+2]['Speed_kmh'].iloc[0]*5/18
#                 #print(f'cur_speed={cur_speed}, next_speed={next_speed}, time={avg_missing_time}, acc={abs(cur_speed - next_speed) / avg_missing_time}')
#                 if (avg_missing_time != 0):
#                     acc[i] = abs(cur_speed - next_speed) / avg_missing_time
                
                
#         start_min = curr_min
#         segment_time = 0
#         zeros = []
#         marker.append(index)

#     cur_speed = row['Speed_kmh']*5/18
#     if (cur_speed == 0):
#         seconds.append(0)
#         zeros.append(index)
#         acc.append(0)
#         continue
#     cur = row['geometry']
#     next = ProcessedData[index+1:index+2]['geometry']
#     time = distance(cur, next)/(cur_speed)
#     seconds.append(time)
#     segment_time = segment_time + time
#     if (index == last_index):
#         acc.append(0)
#     else:
#         next_speed = ProcessedData[index+1:index+2]['Speed_kmh'].iloc[0]*5/18
#         acc.append(abs(cur_speed - next_speed) / time)
    
# # summing time in each minute segment
# sum = 0
# mark = marker.pop(0)
# for i in range(len(seconds)):
#     if (i == mark):
#         sum = 0
#         if (marker):
#             mark = marker.pop(0)
#     sum += seconds[i]
#     seconds[i] = sum

# ProcessedData["seconds"] = seconds
# ProcessedData["acceleration"] = acc
# ProcessedData.to_csv("sa.csv")

    EPISODEID  RECORDID       TIMESTART MODES                    geometry
0          26      5674  9/3/2009 11:50  walk  POINT (-79.79024 43.33742)
1          26      5681  9/3/2009 11:54  stop  POINT (-79.79030 43.33742)
2          26      5690  9/3/2009 11:57   car  POINT (-79.79025 43.33740)
3          26      6009  9/3/2009 12:04  stop  POINT (-79.78155 43.33184)
4          26      6015  9/3/2009 12:08   car  POINT (-79.78169 43.33186)
5          26      6259  9/3/2009 12:12  stop  POINT (-79.79401 43.34516)
6          26      6281  9/3/2009 12:16   car  POINT (-79.79395 43.34526)
7          26      6625  9/3/2009 12:21  stop  POINT (-79.78248 43.35763)
8          26      6668  9/3/2009 13:02   car  POINT (-79.78248 43.35775)
9          26      9938  9/3/2009 13:56  stop  POINT (-79.74953 43.67333)
10         26      9950  9/3/2009 14:03   car  POINT (-79.74969 43.67332)
11         26     10250  9/3/2009 14:08  stop  POINT (-79.74698 43.66366)
12         26     10275  9/3/2009 14:2